In [6]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [2]:
from pathlib import Path

from bisect import bisect_left

In [3]:
lines = Path("data/05.txt").read_text().strip().split("\n")
lines[:10]

['seeds: 2637529854 223394899 3007537707 503983167 307349251 197383535 3543757609 276648400 2296792159 141010855 116452725 5160533 2246652813 49767336 762696372 160455077 3960442213 105867001 1197133308 38546766',
 '',
 'seed-to-soil map:',
 '1024364543 1121869540 764570177',
 '1788934720 0 30748436',
 '710359306 576061773 314005237',
 '1819683156 509305212 66756561',
 '478556776 890067010 231802530',
 '0 30748436 478556776',
 '']

In [4]:
seeds = list(map(int, lines[0].split(":")[-1].split()))
seeds

[2637529854,
 223394899,
 3007537707,
 503983167,
 307349251,
 197383535,
 3543757609,
 276648400,
 2296792159,
 141010855,
 116452725,
 5160533,
 2246652813,
 49767336,
 762696372,
 160455077,
 3960442213,
 105867001,
 1197133308,
 38546766]

In [5]:
seed2soil, soil2fert, fert2water, water2light, light2temp, temp2hum, hum2loc = (
    [],
    [],
    [],
    [],
    [],
    [],
    [],
)
labels = [
    "seed2soil",
    "soil2fert",
    "fert2water",
    "water2light",
    "light2temp",
    "temp2hum",
    "hum2loc",
]

maps = [seed2soil, soil2fert, fert2water, water2light, light2temp, temp2hum, hum2loc]
k = -1
for i, line in enumerate(lines[2:]):
    if "map" in line:
        k += 1
        continue
    if line == "":
        continue
    maps[k].append(list(map(int, line.split())))

for k in range(len(maps)):
    maps[k] = sorted(maps[k], key=lambda t: t[1])

# maps

---

In [7]:
best_min = int(1e19)
for seed in seeds:
    # print(f"SEED {seed}")
    result = seed

    for k in range(len(maps)):
        # print(f"  {labels[k]} [{result}]")
        ranges = maps[k]
        i_best = bisect_left(ranges, result, key=lambda t: t[1])
        for s in range(i_best - 1, i_best + 2):
            if not (0 <= s < len(ranges)):
                continue
            rng = ranges[s]
            if rng[1] <= result < rng[1] + rng[2]:
                result = result - rng[1] + rng[0]
                break
    best_min = min(best_min, result)
    # print(result)
    # print("---------")
best_min

323142486

In [8]:
from tqdm import tqdm

In [10]:
best_min = int(1e19)
for start, rng_len in zip(seeds[::2], seeds[1::2]):
    for seed in tqdm(range(start, start + rng_len)):
        # print(f"SEED {seed}")
        result = seed

        for k in range(len(maps)):
            # print(f"  {labels[k]} [{result}]")
            ranges = maps[k]
            i_best = bisect_left(ranges, result, key=lambda t: t[1])
            for s in range(i_best - 1, i_best + 2):
                if not (0 <= s < len(ranges)):
                    continue
                rng = ranges[s]
                if rng[1] <= result < rng[1] + rng[2]:
                    result = result - rng[1] + rng[0]
                    break
        best_min = min(best_min, result)
        # print(result)
        # print("---------")
best_min

100%|███████████████████████████| 38546766/38546766 [03:06<00:00, 207117.71it/s]


79874951

In [58]:
class Range:
    def __init__(self, min=None, max=None, dx=0):
        if min is None:
            min = float("-inf")
        if max is None:
            max = float("inf")
        self.min = min
        self.max = max
        self.dx = dx

    def __mul__(self, other):
        if other.min > self.max or self.min > other.max:
            return
        return Range(
            min=max(self.min, other.min),
            max=min(self.max, other.max),
            dx=self.dx + other.dx,
        )

    def __sub__(self, other):
        u = self * other
        if u is None:
            return self
        if self.min > u.min and self.max < u.max:
            return (
                Range(self.min, u.min - 1, self.dx),
                Range(u.max + 1, self.max, self.dx),
            )

        if self.min < u.min:
            return Range(self.min, u.min - 1, self.dx)
        else:
            return Range(u.max + 1, self.max, self.dx)

    def __add__(self, other):
        return Range(self.min + other, self.max + other, self.dx)

    def __lt__(self, other):
        if self.min != other.min:
            return self.min < other.min
        return self.max < other.max

    def __str__(self) -> str:
        return f"Range(min={self.min}, max={self.max}, dx={self.dx})"

    def __repr__(self) -> str:
        return str(self)

In [59]:
a = Range(9, 11, 10)
b = Range(10, 15, 10)

print(a * b)
print(a - b)
print(b - a)

Range(min=10, max=11, dx=20)
Range(min=9, max=9, dx=10)
Range(min=12, max=15, dx=10)


In [60]:
a < b

True

In [102]:
A = [Range(1, 5, 999), Range(10, 15, 999)]
B = [Range(3, 6, 1000)]

ia, ib = 0, 0

if A[ia] < B[ib]:
    r = A[ia]
    ia += 1
    from_a = True
else:
    r = B[ib]
    ib += 1
    from_a = False

R = [r]

if from_a:
    if B[ib] * r is not None:
        # merge
        print(f"Merging {r} with {B[ib]} here")
        pass
    else:
        r = B[ib]
        ib += 1
        from_a = False
        R.append(r)
else:
    if A[ia] * r is not None:
        # merge
        print(f"Merging {r} with {A[ia]} here")
        pass
    else:
        r = A[ia]
        ia += 1
        from_a = True
        R.append(r)

Merging Range(min=1, max=5, dx=999) with Range(min=3, max=6, dx=1000) here


---

In [101]:
from tqdm import tqdm

In [108]:
lines = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4""".split(
    "\n"
)

In [111]:
seed2soil

[[50, 98, 2], [52, 50, 48]]

In [116]:
from collections import namedtuple

Range = namedtuple("Range", ["min", "max", "dx"])

In [120]:
def build_ranges(ranges):
    ranges = sorted(ranges, key=lambda t: t[1])
    result = [Range(min=float("-inf"), max=ranges[0][1] - 1, dx=0)]
    for rng in ranges:
        result.append(Range(min=rng[1], max=rng[1] + rng[2] - 1, dx=rng[1] - rng[0]))
    result.append(Range(min=ranges[-1][1] + ranges[-1][2], max=float("inf"), dx=0))
    return result

In [121]:
build_ranges(seed2soil)

[Range(min=-inf, max=49, dx=0),
 Range(min=50, max=97, dx=-2),
 Range(min=98, max=99, dx=48),
 Range(min=100, max=inf, dx=0)]

In [ ]:
def subdivide(rangesA, rangesB):
    ia, ib = 0, 0
    result = []
    if rangesA[ia].min < rangesB[ib].min:
        result.append(Range(min=float("-inf"), max=rangesA[ia].min - 1, dx=0))
        if rangesA[ia].max < rangesB[ib].min:
            result.append(Range(min=rangesA[ia].min, max=rangesA[ia].max, dx=rangesA[ia].dx))
        else:
            result.append(Range(min=rangesA[ia].min, max=rangesB[ib].min, dx=0))